In [5]:
#return True if a stock ticker's most recent news occured in last X amount of time
def get_news_story_time_diff(symbol:str):
    sn = rs.stocks.get_news(symbol=symbol)
    if sn:
        news_time = sn[0]['published_at']
    else:
        return False
    #formatting news story time
    news_time_form = news_time.replace('T', " ")
    news_time_form = news_time_form.replace ('Z', "")
    #formatting current time
    cap_time = datetime.now()
    cap_time_form = cap_time.strftime("%Y-%m-%d %H:%M:%S")
    cap_time_form = str(cap_time_form)
    
    #getting time difference between the two
    d1 = datetime.strptime(cap_time_form, "%Y-%m-%d %H:%M:%S")
    d2 = datetime.strptime(news_time_form, "%Y-%m-%d %H:%M:%S")
    diff = d1-d2
    #1800 is 30 minutes
    if diff.total_seconds() < 1800:
        #then less than 30 minutes since story
        return True
    else:
        return False

In [6]:
def filter_recent_news():
    #download table
    storage_client = storage.Client(project="bright-voltage-248522")
    bucket = storage_client.get_bucket('arys_bucket1')
    blob = bucket.blob("stable_current_stocks")
    data = blob.download_to_filename('stable_current_stocks1.csv')
    data_csv = pd.read_csv('stable_current_stocks1.csv', index_col=0, parse_dates=True)
      
    print("number of tradeable stocks ", len(data_csv))
    #iterate through tickers  
    for index, row in data_csv.iterrows():
        if get_news_story_time_diff(row["symbol"]) == False:
            data_csv = data_csv.drop([index], axis=0)
    
    return data_csv